# Ploidy analysis
## Script to perform ploidy analysis of nuclear, cell type marker and cell cycle marker staining images 

## 1. Load dependencies

In [ ]:
import numpy as np
from skimage import io, color
import pandas as pd
from natsort import os_sorted
import glob
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("dark_background")
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PloidyAnalysis_2D import measure_properties, process_data, process_images, simple_overlay, interactive_overlay

## 2. Get data

In [ ]:
"""
Provide image directories as well as cell type marker ame (mark) and cell cycle marker name (cc)
"""

nuc_img_directories = "/path/to/nuclear/SumIPs/" # nuclear sum projection image directories
nuc_label_directories = "/path/to/reindexed/nuclear/labels/" # nuclear labels directories
nuc_max_directories = "/path/to/nuclear/MaxIPs/" # nuclear max images directories
mark_img_directories = "/path/to/marker/SumIPs/" # cell type marker sum projection images directories
mark_label_directories = "/path/to/reindexed/marker/labels/" # cell type marker labels directories
mark_max_directories = "/path/to/marker/MaxIPs/" # cell type marker max images directories
cc_img_directories = "/path/to/cellcycle/SumIPs/" # cell cycle marker sum image directories
cc_max_directories = "/path/to/cellcycle/MaxIPs/" # cell cylce marker max image directories

mark = "CellTypeMarker"
cc = "CellCycleMarker"

In [ ]:
nuc_images = os_sorted(glob.glob(nuc_img_directories + "*.tif"))
nuc_label_images = os_sorted(glob.glob(nuc_label_directories + "*.tif"))
nuc_max_images = os_sorted(glob.glob(nuc_max_directories + "*.tif"))
mark_images = os_sorted(glob.glob(mark_img_directories + "*.tif"))
mark_label_images = os_sorted(glob.glob(mark_label_directories + "*.tif"))
mark_max_images = os_sorted(glob.glob(mark_max_directories + "*.tif"))
cc_images = os_sorted(glob.glob(cc_img_directories + "*.tif"))
cc_max_images = os_sorted(glob.glob(cc_max_directories + "*.tif"))

## 3. Process data

In [ ]:
processed_data = process_images(nuc_images, mark_images, cc_images, nuc_label_images, mark_label_images, mark, cc)

In [ ]:
processed_data

## 4. Initial data inspection

In [ ]:
simple_overlay(nuc_images[0], nuc_label_images[0])

In [ ]:
simple_overlay(mark_images[0], mark_label_images[0])

In [ ]:
plt.style.use("dark_background")
sns.pairplot(
    processed_data, kind ="hist", height=4,
    vars=[f"{mark}_median_intensity_bgCorr", f"{mark}_CTCF_n", f"{mark}_volume",
          "nuc_median_intensity_bgCorr", "nuc_CTCF_n", "nuc_volume",
          f"{cc}_median_intensity_bgCorr", f"{cc}_CTCF_n"],
    diag_kws= {'color': '#59debf'})

## 5. Exclude cycling cells

In [ ]:
"""
Given a list of image files, the correponding labels files, the desired series, tabular data of label features, alpha value for opacity, low contrast threshold, high contrast threshold, a color mode, the output directory as well as the output file name,
creates an masked overlay with hover information provided for each label.
"""
#interactive_overlay(image_files, labels_files, series, data, mark, cc, label_alpha, contrast_lo, contrast_hi, color_mode, out_dir, out_file_name)
interactive_overlay(cc_max_images, nuc_label_images, 1, mark, cc, processed_data, 0.4, 200, 20000, "default_colormode", "/path/to/out_dir", "ccImage_nucLabels")

In [ ]:
x = processed_data[f'{cc}_median_intensity_bgCorr']

fig = go.Figure()
fig.add_trace(go.Histogram(x=x, nbinsx=700, marker_color='cyan', name='G1'))
fig.update_layout(barmode='overlay',
                  template="plotly_dark",
                  height=800,
                  xaxis_title_text=f'{cc} median fluorescence intensity',
                  yaxis_title_text='Count',)
fig.update_traces(opacity=0.6)

fig.show()

In [ ]:
y = processed_data['nuc_volume']
x = processed_data[f'{cc}_median_intensity_bgCorr']

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, marker_color='cyan', mode="markers"))

fig.update_layout(template="plotly_dark",
                  height=800,
                  xaxis_title_text=f'{cc} median fluorescence intensity',
                  yaxis_title_text='Nuclear volume',)
fig.update_traces(opacity=0.6)
#fig.update_yaxes(type="log")

fig.show()

### Categorize cells into "G1" and "G2" based on cell cycle marker expression and nuclear volume

In [ ]:
def categorise_cell_cycle(row):
    if row[f'{cc}_median_intensity_bgCorr'] > 8000:
        return 'G2'
    else:
        return 'G1'

In [ ]:
processed_data['cell_cycle'] = processed_data.apply(lambda row: categorise_cell_cycle(row), axis=1)

In [ ]:
processed_data["cell_cycle"].value_counts()

In [ ]:
processed_data

In [ ]:
"""
Given a list of image files, the correponding labels files, the desired series, tabular data of label features, alpha value for opacity, low contrast threshold, high contrast threshold, a color mode, the output directory as well as the output file name,
creates an masked overlay with hover information provided for each label.
"""
#interactive_overlay(image_files, labels_files, series, data, mark, cc, label_alpha, contrast_lo, contrast_hi, color_mode, out_dir, out_file_name)
interactive_overlay(cc_max_images, nuc_label_images, 1, mark, cc, processed_data, 0.4, 200, 20000, "cell_cycle", "/path/to/out_dir", "ccImage_nucLabels_cellCycle")

In [ ]:
x0 = processed_data[processed_data["cell_cycle"]=="G1"]["nuc_CTCF_n"]
x1 = processed_data[processed_data["cell_cycle"]=="G2"]["nuc_CTCF_n"]

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0, nbinsx=400, marker_color='cyan', name='G1'))
fig.add_trace(go.Histogram(x=x1, nbinsx=400, marker_color='magenta', name='G2'))

fig.update_layout(barmode='overlay',
                  template="plotly_dark",
                  height=800,
                  xaxis_title_text='Nuclear total DAPI fluorescence',
                  yaxis_title_text='Count',)
fig.update_traces(opacity=0.6)

fig.show()

In [ ]:
y0 = processed_data[processed_data["cell_cycle"]=="G1"]["nuc_CTCF_n"]
y1 = processed_data[processed_data["cell_cycle"]=="G2"]["nuc_CTCF_n"]

x0 = processed_data[processed_data["cell_cycle"]=="G1"][f"{cc}_median_intensity_bgCorr"]
x1 = processed_data[processed_data["cell_cycle"]=="G2"][f"{cc}_median_intensity_bgCorr"]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x0, y=y0, marker_color='cyan', name='G1', mode="markers"))
fig.add_trace(go.Scatter(x=x1, y=y1, marker_color='magenta', name='G2', mode="markers"))

fig.update_layout(barmode='overlay',
                  template="plotly_dark",
                  height=800,
                  xaxis_title_text=f'{cc} median fluorescence intensity',
                  yaxis_title_text='Nuclear total DAPI fluorescence',)
fig.update_traces(opacity=0.6)
fig.update_xaxes(type="log")


fig.show()

## 6. Categorize cells by ploidy

In [ ]:
def categorise_ploidy(row):
    if row['cell_cycle'] == "G1" and row['nuc_CTCF_n'] <= 8.5:
        return '2N'
    elif row['cell_cycle'] == "G1" and row['nuc_CTCF_n'] > 8.5:
        return '4N'
    else:
        return 'NA'

In [ ]:
processed_data['ploidy'] = processed_data.apply(lambda row: categorise_ploidy(row), axis=1)

In [ ]:
processed_data["ploidy"].value_counts()

In [ ]:
"""
Given a list of image files, the correponding labels files, the desired series, tabular data of label features, alpha value for opacity, low contrast threshold, high contrast threshold, a color mode, the output directory as well as the output file name,
creates an masked overlay with hover information provided for each label.
"""
#interactive_overlay(image_files, labels_files, series, data, label_alpha, contrast_lo, contrast_hi, color_mode, out_dir, out_file_name)
interactive_overlay(cc_max_images, nuc_label_images, 1, mark, cc, processed_data, 0.4, 200, 20000, "ploidy", "/path/to/out_dir", "ccImage_nucLabels_ploidy")

## 7. Determine type of polyploid cells

In [ ]:
x0 = processed_data[f"{mark}_median_intensity_bgCorr"]

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0, nbinsx=500, marker_color='cyan', name='G1'))

fig.update_layout(barmode='overlay',
                  template="plotly_dark",
                  height=800,
                  xaxis_title_text=f'{mark} total fluorescence',
                  yaxis_title_text='Count',)
fig.update_traces(opacity=0.6)
#fig.update_xaxes(type="log")


fig.show()

In [ ]:
x0 = processed_data[f"{mark}_median_intensity_bgCorr"]
    
y0 =  processed_data[f"{mark}_volume"]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x0, y=y0, marker_color='cyan', name='G1', mode="markers"))

fig.update_layout(barmode='overlay',
                  template="plotly_dark",
                  height=800,
                  yaxis_title_text=f'{mark} volume',
                  xaxis_title_text=f'{mark} total fluorescence',)
fig.update_traces(opacity=0.6)

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")

fig.show()

### Categorize cells by cell type marker expression

In [ ]:
def categorise_cell_type(row):
    if row[f'{mark}_median_intensity_bgCorr'] >= 6000:
        return f'{mark}_positive'
    else:
        return f'{mark}_negative'

In [ ]:
processed_data['cell_type'] = processed_data.apply(lambda row: categorise_cell_type(row), axis=1)

In [ ]:
processed_data["cell_type"].value_counts()

In [ ]:
processed_data

In [ ]:
x0 = processed_data[(processed_data["cell_type"]==f"{mark}_positive") & (processed_data["ploidy"]=="2N") | (processed_data["ploidy"]=="4N")]["nuc_CTCF_n"]
x1 = processed_data[(processed_data["cell_type"]==f"{mark}_positive") & (processed_data["ploidy"]=="NA")]["nuc_CTCF_n"]
x2 = processed_data[processed_data["cell_type"]==f"{mark}_negative"]["nuc_CTCF_n"]

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0, nbinsx=300, marker_color='cyan', name=f'{mark}_positive G1'))
fig.add_trace(go.Histogram(x=x1, nbinsx=300, marker_color='yellow', name=f'{mark}_positive G2'))
fig.add_trace(go.Histogram(x=x2, nbinsx=300, marker_color='magenta', name=f'{mark}_negative'))

fig.update_layout(barmode='overlay',
                  template="plotly_dark",
                  height=800,
                  xaxis_title_text='Nuclear total fluorescence',
                  yaxis_title_text='Count',)
fig.update_traces(opacity=0.5)

fig.show()

In [ ]:
diploid = processed_data[processed_data['ploidy'] == '2N']['cell_type'].value_counts()
tetraploid = processed_data[processed_data['ploidy'] == '4N']['cell_type'].value_counts()
diploid100 = pd.Series(data={f"{mark}_positive": diploid.values[0]/(diploid.values[0]+tetraploid.values[0])*100, f"{mark}_negative": diploid.values[1]/(diploid.values[1]+tetraploid.values[1])*100}, index=[f'{mark}_positive', f'{mark}_negative'])
tetraploid100 = pd.Series(data={f"{mark}_positive": tetraploid.values[0]/(diploid.values[0]+tetraploid.values[0])*100, f"{mark}_negative": tetraploid.values[1]/(diploid.values[1]+tetraploid.values[1])*100}, index=[f'{mark}_positive', f'{mark}_negative'])

trace1 = go.Bar(
    x=diploid.index,
    y=diploid.values,
    name='2N',
    marker_color='cyan'
)
trace2 = go.Bar(
    x=tetraploid.index,
    y=tetraploid.values,
    name='4N',
    marker_color='magenta'
)

trace3 = go.Bar(
    x=diploid100.index,
    y=diploid100.values,
    name='2N',
    marker_color='cyan',
    showlegend=False
)
trace4 = go.Bar(
    x=tetraploid100.index,
    y=tetraploid100.values,
    name='4N',
    marker_color='magenta',
    showlegend=False
)

data = [trace1, trace2]

fig = make_subplots(rows=1,
                    cols=2,
                    subplot_titles=("Ploidy absolute", "Ploidy relative"),
                    horizontal_spacing=0.3
                   )

fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=1)
fig.add_trace(trace3, row=1, col=2)
fig.add_trace(trace4, row=1, col=2)

fig.update_xaxes(title_text="Cell Type", row=1, col=1)
fig.update_xaxes(title_text="Cell Type", row=1, col=2)
fig.update_yaxes(title_text="Cell Count", row=1, col=1)
fig.update_yaxes(title_text="% Cells", row=1, col=2)

fig.update_layout(barmode='stack',
                  template="plotly_dark",
                  height=800,
                  width=1000
)
fig.show()

In [ ]:
"""
Given a list of image files, the correponding labels files, the desired series, tabular data of label features, alpha value for opacity, low contrast threshold, high contrast threshold, a color mode, the output directory as well as the output file name,
creates an masked overlay with hover information provided for each label.
"""
#interactive_overlay(image_files, labels_files, series, data, label_alpha, contrast_lo, contrast_hi, color_mode, out_dir, out_file_name)
interactive_overlay(mark_max_images, mark_label_images, 1, mark, cc, processed_data, 0.4, 200, 20000, "cell_type", "/path/to/out_dir", "markImage_markLabels_cellType")